In [0]:
%sql
WITH customer_cte (customer_nk, first_name, last_name, phone, country, email_id) 
AS (SELECT * FROM csv.`/Volumes/source_files/default/files/Customer_1.csv`
UNION ALL
SELECT * FROM csv.`/Volumes/source_files/default/files/Customer_2.csv`
UNION ALL
SELECT * FROM csv.`/Volumes/source_files/default/files/Customer_3.csv`),
customer_cte_versioned AS 
(SELECT 
  ROW_NUMBER() OVER (PARTITION BY customer_nk ORDER BY customer_nk) 
  customer_version, customer_nk , first_name, last_name, phone, country, email_id,
  TRY_CAST(REPLACE(customer_nk, 'CUST', '') AS INT) AS customer_nk_num 
FROM customer_cte),
customer_cte_filtered AS (SELECT * FROM customer_cte_versioned WHERE customer_nk_num IS NOT NULL)
INSERT INTO case_study_de.customer_bronze (
  customer_nk,
  first_name,
  last_name,
  phone,
  country,
  email_id ,
  custmer_nk_num,
  customer_version
)
SELECT customer_nk, first_name, last_name, phone, country, email_id, customer_nk_num, customer_version   
FROM customer_cte_filtered;
